In [1]:
import numpy as np
from numpy import array as arr

In [2]:
import sys
import socket
if socket.gethostname() == 'Birks-iMac.fritz.box':
    sys.path.append('/Users/Birk/Documents/Programmieren/Python/Eigener Code')
    sys.path.append('/Users/Birk/Documents/Programmieren/GitHub/experiment-code/pap')
else:
    sys.path.append('/Home/ab227/linux/Eigener Code')

    
from pap import summen_fehler
from pap import _rundung, _negativ_wird_null, _istbool

# pap.resultat() - Entwicklung

In [3]:
'''
Benötigt:
* numpy als np
* np.array() als arr()

* pap.summen_fehler()
* pap._rundung()
* pap._istbool()
* pap._negativ_wird_null()
'''


def resultat(titel, werte, einheit = '', faktor = 1, nachkommastellen = None, rel_fehler = False):
    '''
    Printet ein schön formatiertes Ergebnis mit 
    Titel, definierter Präzision, evt. +/- Fehler, Einheit und evt. relativen Fehler.
    
    
    Argumente
    ---------
    titel : str
    
    werte : number_like, np.ndarray (1D, mit number_like Elementen)
        Darf die Formen haben 
        ein_wert, np.array([ein_wert]),
        np.array([ein_wert, sein_fehler]) oder 
        np.array([ein_wert, sys_fehler, stat_fehler]).
    
    einheit : str
    
    faktor : number_like, optional
        Ermöglicht Anpassung an Größenordnung und Einheit.
    
    nachkommastellen : int, optional
        Siehe "Rundung des Ergebnisses".
    
    rel_fehler : bool, number_like, optional
        darf False sein     (aus),
             True sein      (wenn Fehler vorhanden und ein_wert != 0,
                             dann wird dieser als sein_fehler / ein_wert bzw. als
                             (quadratische summe von sys- und stat_fehler) / ein_wert
                             berechnet.), oder
             eine Zahl sein (Einheit Prozent; diese wird dann direkt angegeben)
    
    
    Beispiele
    ---------
    >>> pap.resultat('Abweichung', 0.30304, '%', faktor = 1e2, nachkommastellen = 0)
      Abweichung: 30 %
    
    >>> pap.resultat('Arbeit', np.array([3.35e6, 0.46e6]), 'MJ', faktor = 1e-6)
      Arbeit: 3.4 +/- 0.5 MJ
    
    >>> pap.resultat('Beschleunigung', np.array([9.8493, 0.0424, 0.1235]), 'm/s^2')
      Beschleunigung: 9.85 +/- 0.04(sys) +/- 0.12 (stat) m/s^2
    
    
    mit relativem Fehler:
    
    >>> pap.resultat('Arbeit           ', np.array([3.3423, 0.4679]), 'J', rel_fehler = True)
      Arbeit           : 342 +/- 5 J   (1.6 %)
    
    >>> pap.resultat('Fehler der Arbeit', 0.4679, 'J', rel_fehler = 1.6)
      Fehler der Arbeit: 5 J   (1.6 %)
                    
    
    Rundung der Ergebnisse
    ----------------------
      * Gibt man die Zahl der Nachkommastellen manuell an, so wird nach ihr gerundet.
        Dabei sind auch negative Werte erlaubt. Bei -1 z.B. wird 123 -> 120 gerundet.
      * Ansonsten wird automatisch nach der ersten signifikanten Stelle des größten
        Fehlers gerundet. z.B. [-123.33, -5.0, 0.02] wird zu "-123.3 +/- 5.0 +/- 0.0"
        Ist der Betrag dieser Stelle aber < 4, dann wird auf zwei signifikante 
        Stellen gerundet, d.h. 0.436 -> "0.4", aber 0.0392 -> "0.39"
      * Gibt es kein Fehler, oder sind diese alle 0, dann wird automatisch
        nachkommastellen = 16 gesetzt.
      * Der relative Fehler wird stehts auf zwei signifikante Stellen gerundet.
    '''
    
    
    
    # Überprüfen und Korrigierung von werte
    if type(werte) != np.ndarray:   # Falls werte nur eine Zahl ist, wird sie zum Array gemacht.
        werte = arr([werte])
    if len(werte) > 3:
        print('Zu viele Elemente in werte! >:(')
    
    werte = werte * faktor   # Umrechnung der Resultate auf gewünschte Einheit oder Größenordnung
    if len(werte) > 1:
        werte[1:] = np.abs(werte[1:])   # Keine negativen Fehlerangaben
        fehler = werte[1:]
        
        
    # eventuelles Erstellen eines relativen Fehlers
    if _istbool(rel_fehler, True):
        if werte[0] == 0:   # Um einen divide-by-zero Fehler zu vermeiden.
            rel_fehler = False
        elif len(werte) == 1:   # Um einen divide-by-nothing Fehler zu vermeiden.
            rel_fehler = False
        elif len(werte) == 2:
            rel_fehler = fehler[0] / werte[0] * 100   # [%], relativer Fehler
        elif len(werte) == 3:
            rel_fehler = summen_fehler(fehler) / werte[0] * 100   # [%], relativer Fehler des Gesamtfehlers
    
    if not _istbool(rel_fehler, False):
        # falls rel_fehler eine Zahl ist, wird mit dieser weitergerechnet, bei False nicht. 
        # In anderen Fällen ensteht hier gleich ein Fehler.
        
        # Rundung auf 2 signifikanten Stellen 
        rel_fehler = np.abs(np.float64(rel_fehler))
        if rel_fehler != 0:
            größenordnung = np.int(np.floor(np.log10(rel_fehler)))
            signifikante_stellen = 2
            präzision = -größenordnung + signifikante_stellen - 1
        else:
            präzision = 2
        rel_fehler = _rundung(rel_fehler, präzision)
        
        # Vorbereitung des Relativer-Fehler-Strings
        präzision = _negativ_wird_null(präzision)   # Keine negativen Werte für format(prec=) erlaubt.
        if rel_fehler != 0:
            if größenordnung > -5:   # Sehr kleine Werte sehen besser mit e aus.
                rel_fehler_string = '   ({:.{prec}f} %)'.format(rel_fehler, prec = präzision)
            else:
                rel_fehler_string = '   ({:.1f}e{} %)'.format(rel_fehler * 10**(-größenordnung),
                                                              größenordnung)
        else:
            rel_fehler_string = f'   ({rel_fehler} %)'
    
    else:
        rel_fehler_string = ''
        
    
    # Bestimmung der Präzision des Ergebnisses   
    if nachkommastellen == None:
        if len(werte) > 1:
            größter_fehler = np.max(fehler)
            if größter_fehler != 0:
                größenordnung = np.int(np.floor(np.log10(größter_fehler)))
                signifikante_stellen = (1 if größter_fehler / 10**größenordnung >= 4.0
                                        else 2)   # Hier der 4.0-Cutoff
                nachkommastellen = -größenordnung + signifikante_stellen - 1
            else:
                nachkommastellen = 8   # Da Fehler = 0   
        else:
            nachkommastellen = 8   # Da kein Fehler vorhanden.
    
    
    # Rundung entsprechend der signifikante Stelle oder der eigenen Vorgabe
    wertepaar = _rundung(werte, nachkommastellen)
    nachkommastellen = _negativ_wird_null(nachkommastellen)   # Keine negativen Werte für 
                                                              # format(prec=) erlaubt.
    
    
    # Print des Ergebnis-Strings
    if len(werte) == 1:
        print(titel + ': {:.{prec}f} {}{}'
              .format(*wertepaar, einheit, rel_fehler_string, prec = nachkommastellen))
    elif len(werte) == 2:
        print(titel + ': {:.{prec}f} +/- {:.{prec}f} {}{}'
              .format(*wertepaar, einheit, rel_fehler_string, prec = nachkommastellen))
    elif len(werte) == 3:
        print(titel + ': {:.{prec}f} +/- {:.{prec}f}(sys) +/- {:.{prec}f}(stat) {}{}'
              .format(*wertepaar, einheit, rel_fehler_string, prec = nachkommastellen))

# pap.resultat Tests

In [4]:
resultat('Höhe', 81.209384, 'm')
resultat('Höhe', arr([81.209384]), 'm', rel_fehler = False)
resultat('Höhe', arr([81.209384]), 'm', rel_fehler = True)
resultat('Höhenfehler', arr([-81.209384]), 'm', rel_fehler = 38.3945)
resultat('Höhenfehler', arr([-81.209384]), 'm', rel_fehler = -38.3945)
resultat('Höhenfehler', arr([-81.209384]), 'm', rel_fehler = -98.3945)
resultat('Höhenfehler', arr([-81.209384]), 'm', rel_fehler = -122.39)
resultat('Höhenfehler', arr([-81.209384]), 'm', rel_fehler = -3839.45)
#resultat('Höhenfehler', arr([81.209384]), 'm', rel_fehler = arr([1, 0]))   # Fehlermeldung, wie auch bei None und strings
print('')
resultat('Höhe', arr([81.209384, -4.049378]), 'm')
resultat('Höhe', arr([81.209384, 4.049378]), 'm', rel_fehler = False)
resultat('Höhe', arr([0, -4.049378]), 'm', rel_fehler = True)
resultat('Höhe', arr([81.209384, 4.049378]), 'm', rel_fehler = 0)
resultat('Höhe', arr([81.209384, 4.049378]), 'm', rel_fehler = 0.0)
resultat('Höhe', arr([81.209384, 4.049378]), 'm', rel_fehler = 1)
resultat('Höhe', arr([81.209384, -4.049378]), 'm', rel_fehler = True)
resultat('Höhe', arr([81.209384, 0]), 'm', rel_fehler = True)
resultat('Höhe', arr([81.209384, 0.0]), 'm', rel_fehler = True)
resultat('Höhe', arr([81.209384, 0.0000]), 'm', rel_fehler = True)
resultat('Höhe', arr([81.209384, 0.0000000001]), 'm', rel_fehler = True)
resultat('Höhe', arr([81.209384, 0]), 'm', rel_fehler = 0.00000000010)
print('')
resultat('Höhe', arr([81.209384, 4.049378, -2.38458]), 'm', rel_fehler = False)
resultat('Höhe', arr([81.209384, 4.049378, -2.38458]), 'm', rel_fehler = True)
resultat('Höhe', arr([81.209384, 0, -2.38458]), 'm', rel_fehler = True)
resultat('Höhe', arr([81.209384, 4.049378, -2.38458]), 'm', rel_fehler = True)
resultat('Höhe', arr([81.209384, 0, 0]), None, rel_fehler = True)
resultat('Höhe', arr([81.209384, 0, 0]), 'm', rel_fehler = 0)
resultat('Höhe', arr([81.209384, 0, 0]), 'm', rel_fehler = 1e-4)
resultat('Höhe', arr([81.209384, 0, 0]), 'm', rel_fehler = 1.2784e-4)
resultat('Höhe', arr([81.209384, 0, 0]), 'm', rel_fehler = 0.97273e-4)
resultat('Höhe', arr([81.209384, 0, 0]), 'm', rel_fehler = 1e-5, nachkommastellen = 2)
print('')
resultat('Hallo', arr([0.33094824, -0.004223, 0.000098434]), 'Würstchen', faktor = 1e3, nachkommastellen = 1, 
         rel_fehler = True)

Höhe: 81.20938400 m
Höhe: 81.20938400 m
Höhe: 81.20938400 m
Höhenfehler: -81.20938400 m   (38 %)
Höhenfehler: -81.20938400 m   (38 %)
Höhenfehler: -81.20938400 m   (98 %)
Höhenfehler: -81.20938400 m   (120 %)
Höhenfehler: -81.20938400 m   (3800 %)

Höhe: 81 +/- 4 m
Höhe: 81 +/- 4 m
Höhe: 0 +/- 4 m
Höhe: 81 +/- 4 m   (0.0 %)
Höhe: 81 +/- 4 m   (0.0 %)
Höhe: 81 +/- 4 m   (1.0 %)
Höhe: 81 +/- 4 m   (5.0 %)
Höhe: 81.20938400 +/- 0.00000000 m   (0.0 %)
Höhe: 81.20938400 +/- 0.00000000 m   (0.0 %)
Höhe: 81.20938400 +/- 0.00000000 m   (0.0 %)
Höhe: 81.20938400000 +/- 0.00000000010 m   (1.2e-10 %)
Höhe: 81.20938400 +/- 0.00000000 m   (1.0e-10 %)

Höhe: 81 +/- 4(sys) +/- 2(stat) m
Höhe: 81 +/- 4(sys) +/- 2(stat) m   (5.8 %)
Höhe: 81.2 +/- 0.0(sys) +/- 2.4(stat) m   (2.9 %)
Höhe: 81 +/- 4(sys) +/- 2(stat) m   (5.8 %)
Höhe: 81.20938400 +/- 0.00000000(sys) +/- 0.00000000(stat) None   (0.0 %)
Höhe: 81.20938400 +/- 0.00000000(sys) +/- 0.00000000(stat) m   (0.0 %)
Höhe: 81.20938400 +/- 0.00000000(sys